https://github.com/looker-open-source/sdk-codegen/tree/main/python/looker_sdk


---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install python-pptx
!pip install python-docx
!pip install looker_sdk



In [ ]:
from pptx import Presentation
from pptx.util import Cm

from docx import Document
from docx.shared import Inches

import PIL.Image as Image
from io import BytesIO
import os
import looker_sdk




In [ ]:
os.environ["LOOKERSDK_BASE_URL"] = "" #If your looker URL has .cloud in it (hosted on GCP), do not include :19999 (ie: https://your.cloud.looker.com).
os.environ["LOOKERSDK_API_VERSION"] = "3.1" #3.1 is the default version. You can change this to 4.0 if you want.
os.environ["LOOKERSDK_VERIFY_SSL"] = "true" #Defaults to true if not set. SSL verification should generally be on unless you have a real good reason not to use it. Valid options: true, y, t, yes, 1.
os.environ["LOOKERSDK_TIMEOUT"] = "120" #Seconds till request timeout. Standard default is 120.

#Get the following values from your Users page in the Admin panel of your Looker instance > Users > Your user > Edit API keys. If you know your user id, you can visit https://your.looker.com/admin/users/<your_user_id>/edit.
os.environ["LOOKERSDK_CLIENT_ID"] =  "" #No defaults.
os.environ["LOOKERSDK_CLIENT_SECRET"] = "" #No defaults. This should be protected at all costs. Please do not leave it sitting here, even if you don't share this document.

print("All environment variables set.")

In [ ]:
sdk = looker_sdk.init40()
print('Looker SDK 4.0 initialized successfully.')

In [ ]:
my_user = sdk.me()

#Output is an instance of the User model, but can also be read like a python dict. This applies to all Looker API calls that return Models.
#Example: The following commands return identical output. Feel free to use whichever style is more comfortable for you.

print(my_user.first_name) #Model dot notation


folder_id="522"

In [ ]:
pptx = Presentation()
for idx, layout in enumerate(pptx.slide_layouts):
    print(idx, layout.name)
title_only_layout = pptx.slide_layouts[5]

for i,look in enumerate(sdk.folder_looks(folder_id)):
  print(look.id, look.title)
  look_request = {

        "look_id": look.id, 

        "result_format": 'png', 

        "image_width": 960, 

        "image_height": 540

    }
  
  try:
    image1 = sdk.run_look(**look_request)
    image = Image.open(BytesIO(image1))
    image_file = ''.join([str(look.id), '.png'])
    image.save(image_file) 
  except:
    print(f'Look failed {look.id}: {look.title}')
    image_file = None
  
  pptx.slides.add_slide(title_only_layout)
  pptx.slides[i].shapes.title.text = f''+look.title
  try:
    pptx.slides[i].shapes.add_picture(image_file, Cm(2), Cm(5), width=Cm(20)) # image, left, top, width
  except:
    print('Failed to add image to slide')
pptx.save('demo.pptx')

In [ ]:

document = Document()
document.add_heading("Insights on inventory Items", level=0)


for i,look in enumerate(sdk.folder_looks(folder_id)):
  print(look.id, look.title)
  look_request = {

        "look_id": look.id, 

        "result_format": 'png', 

        "image_width": 960, 

        "image_height": 540

    }
  
  try:
    image1 = sdk.run_look(**look_request)
    image = Image.open(BytesIO(image1))
    image_file = ''.join([str(look.id), '.png'])
    image.save(image_file) 
  except:
    print(f'Look failed {look.id}: {look.title}')
    image_file = None

  document.add_heading(look.title, level=1)
  document.add_picture(image_file, width=Inches(6))
  document.add_page_break()



document.save('demo.docx')


In [ ]:
arr=[]
for i,items in enumerate(sdk.all_folders()):
  if len(items.name)==0:
    continue
  temp={"folder_id":items.id,"folder_name":items.name}
  looks=[ look.title for look in sdk.folder_looks(items.id)]
  if len(looks)==0:
    continue
  temp["looks_list"]=looks
  arr.append(temp)
print(arr)

  


In [ ]:
for i,items in enumerate(sdk.all_folders()):
  if items.id =="522":
    print(items)
    break